In [0]:
!pip install -q tensorflow-hub

In [0]:
import tensorflow_hub as hub
import tensorflow as tf

In [0]:
ELMO = "https://tfhub.dev/google/elmo/2"
NNLM = "https://tfhub.dev/google/nnlm-en-dim128/1"

In [0]:
def embed(model_name, sentences):
    if model_name == "elmo":
        elmo = hub.Module(ELMO, trainable=True)
        executable = elmo(
            sentences,
            signature="default",
            as_dict=True)["elmo"]

    elif model_name == "nnlm":
        nnlm = hub.Module(NNLM)
        executable = nnlm(sentences)

    else:
        raise NotImplementedError

    return execute(executable)


def execute(tensor):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())

        return sess.run(tensor)

In [0]:
def word_to_sentence(embeddings):
    return embeddings.sum(axis=1)

def get_embeddings_elmo_nnlm(sentences):
    return word_to_sentence(embed("elmo", sentences)), embed("nnlm", sentences)
  
def cos_sim(a, b):
    return np.inner(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

In [35]:
import numpy as np


if __name__ == "__main__":
    sentence1, sentence2 = ['people read the book', 'the book people read']

    # Get embeddings corresponding to each sentences
    results_elmo, results_nnlm = get_embeddings_elmo_nnlm([sentence1, sentence2])

    print("[Cosine Similarity]")
    print("\"{}\" vs \"{}\"".format(sentence1, sentence2))
    print("ELMo:", cos_sim(results_elmo[0], results_elmo[1]))
    print("NNLM:", cos_sim(results_nnlm[0], results_nnlm[1]))

[Cosine Similarity]
"people read the book" vs "the book people read"
ELMo: 0.83875865
NNLM: 1.0
